In [1]:
import torch
import torch.nn as nn

#First Model Linear Regression

In [2]:
#fake data
train_X = torch.randn(100,1) #random numbers
train_Y = 3* train_X + 5 + torch.randn(100,1) * 0.5 #Linear function with limited randomness
#function 3* x_train + 5 + randomness

In [3]:
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression,self).__init__()
        self.linear = nn.Linear(1,1) # takes one input produces one output
    #In pytorch you write your own forward functions
    def forward(self, x):
        return self.linear(x)

In [9]:
#Running the Linear Model ("Driver Code")
modelFFNN = LinearRegression()
lossFn = nn.MSELoss() #common also name criterion
optimizer = torch.optim.SGD(modelFFNN.parameters(), lr=0.01)
for epoch in range(150):
    prediction = modelFFNN(train_X)
    loss = lossFn(prediction, train_Y)
    optimizer.zero_grad() # you have to zero the gradients 
    loss.backward() # back propagation 
    optimizer.step() # Learning step 
    if epoch %10 == 0:
        print(f"Epoch:{epoch}, Training Loss:{loss.item()}")

Epoch:0, Training Loss:32.0713005065918
Epoch:10, Training Loss:22.183534622192383
Epoch:20, Training Loss:15.401761054992676
Epoch:30, Training Loss:10.739148139953613
Epoch:40, Training Loss:7.525900840759277
Epoch:50, Training Loss:5.306326866149902
Epoch:60, Training Loss:3.7696421146392822
Epoch:70, Training Loss:2.7033796310424805
Epoch:80, Training Loss:1.9619348049163818
Epoch:90, Training Loss:1.4452848434448242
Epoch:100, Training Loss:1.084554672241211
Epoch:110, Training Loss:0.8322069048881531
Epoch:120, Training Loss:0.6553540229797363
Epoch:130, Training Loss:0.5311945676803589
Epoch:140, Training Loss:0.4438852369785309


In [14]:
for layer in modelFFNN.children():
    if isinstance(layer, nn.Linear):
        print(f"Model function = x * {layer.state_dict()['weight']} + {layer.state_dict()['bias']}")


Model function = x * tensor([[2.7393]]) + tensor([4.7431])


In [39]:
#Feed forward Neural Network Classification problem
from sklearn.datasets import make_moons
import numpy as np
#generating data using SKlearn sample datasets 
X,Y = make_moons(n_samples=2000, noise=0.1)
train_X = torch.tensor(X, dtype=torch.float32)
train_Y = torch.tensor(Y, dtype=torch.int64)
test_x, test_y = make_moons(n_samples=400, noise=0.1)
test_X = torch.tensor(test_x, dtype=torch.float32)
test_Y = torch.tensor(test_y, dtype=torch.int64)
print(train_Y)

tensor([1, 1, 0,  ..., 1, 0, 0])


In [40]:
#Simple FFNN
class FFNN(nn.Module):
    def __init__(self,dimenisonalEmbedding):
        super(FFNN, self).__init__()
        self.layer1 = nn.Linear(2,dimenisonalEmbedding)
        self.layer2 = nn.Linear(dimenisonalEmbedding,2)
    def forward(self,x):
        x = torch.relu(self.layer1(x))
        return self.layer2(x)

In [47]:
modelFFNN = FFNN(100)
lossFnCE = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelFFNN.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
    prediction = modelFFNN(train_X)
    loss = lossFnCE(prediction, train_Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch %100 == 0:
        print(f"Epoch:{epoch}, Training Loss:{loss.item()}")

Epoch:0, Training Loss:0.6759788990020752
Epoch:100, Training Loss:0.021177159622311592
Epoch:200, Training Loss:0.00737897539511323
Epoch:300, Training Loss:0.004268783610314131
Epoch:400, Training Loss:0.0029220113065093756
Epoch:500, Training Loss:0.002167722675949335
Epoch:600, Training Loss:0.0017013741889968514
Epoch:700, Training Loss:0.0013849803945049644
Epoch:800, Training Loss:0.00115487992297858
Epoch:900, Training Loss:0.000980548677034676


In [48]:
predictionTest = modelFFNN(test_X)
testLoss = lossFnCE(predictionTest, test_Y)
print(f"Testing loss = {testLoss.item()}")

Testing loss = 0.0030432736966758966


In [49]:
import torchvision
import torchvision.transforms as transforms

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

5.0%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




35.8%

100.0%
100.0%

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [50]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.fc1 = nn.Linear(16*26*26, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(-1, 16*26*26)
        return self.fc1(x)

In [52]:
model = SimpleCNN()
lossFnCE = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(5):
    for images, labels in train_loader:
        prediction = model(images)
        loss = lossFnCE(prediction, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch+1}, training loss = {loss.item()}")

epoch 0, training loss = 0.032170966267585754
epoch 1, training loss = 0.011517233215272427
epoch 2, training loss = 0.07380864769220352
epoch 3, training loss = 0.18718363344669342
epoch 4, training loss = 0.017924603074789047
